In [1]:
import os
import codecs
import pandas as pd
from utils.helpers import adding_stanford_nlp_groups_NER_to_stop_words, removing_stanford_nlp_groups_NER_from_stop_words, punct_space_stop, line_review, lemmatized_sentence_corpus

essays = pd.read_csv('./data/prepped_essays_df.csv')

In [2]:
essays = essays[essays['essay_set'] == 1]

In [3]:
len(essays)

1783

In [4]:
essays.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,prompt,has_source_material,source_text,grade_7,grade_8,grade_10
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,NaN,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4.0,3.0,NaN,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5.0,5.0,NaN,10.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0
4,5,1,"Dear @LOCATION1, I know having computers has a...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"More and more people use computers, but not ev...",0,NaN,0,1,0


In [5]:
essays.dropna(axis=1, how='all', inplace=True)

In [6]:
essays.head(1)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,prompt,has_source_material,grade_7,grade_8,grade_10
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,8.0,"More and more people use computers, but not ev...",0,0,1,0


In [7]:
essays.iloc[0, 2]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [8]:
intermediate_directory = os.path.join('./data/intermediate')

essay_set1_txt_filepath = os.path.join(intermediate_directory,
                                   'essay_set1_text_all.txt')

In [9]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:
    
    essay_count = 0

    # create & open a new file in write mode
    with codecs.open(essay_set1_txt_filepath, 'w', encoding='utf_8') as essay_set1_txt_file:

        # loop through all essays in the dataframe
        for row in essays.itertuples():

            # write the essay as a line in the new file and escape newline characters in the original essays
            essay_set1_txt_file.write(row.essay.replace('\n', '\\n') + '\n')
            essay_count += 1

    print('Text from {:,} essays written to the new txt file.'.format(essay_count))
    
else:
    
    with codecs.open(essay_set1_txt_filepath, encoding='utf_8') as essay_set1_txt_file:
        for essay_count, line in enumerate(essay_set1_txt_file):
            pass
        
    print('Text from {:,} essays in the txt file.'.format(essay_count + 1))

Text from 1,783 essays written to the new txt file.
CPU times: user 15.7 ms, sys: 4.24 ms, total: 20 ms
Wall time: 151 ms


#### Text Preprocessing with spaCy

In [10]:
import spacy
import itertools as it

nlp = spacy.load('en_core_web_md')

In [11]:
test_essay = essays.iloc[0, 2]

In [12]:
test_essay

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [13]:
%%time
parsed_essay = nlp(test_essay)

CPU times: user 81 ms, sys: 14.1 ms, total: 95.1 ms
Wall time: 114 ms


In [14]:
for num, sentence in enumerate(parsed_essay.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble!

Sentence 2:
Thing about!

Sentence 3:
Dont you think so?

Sentence 4:
How would you feel if your teenager is always on the phone with friends!

Sentence 5:
Do you ever time to chat with your friends or buisness partner about things.

Sentence 6:
Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect.

Sentence 7:
Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it.

Sentence 8:
How did you learn about other countrys

Sentence 9:
/states outside of yours?

Sentence 10:
Well I have by computer/internet

Sentence 11:
, it's a new way to 

In [15]:
for num, entity in enumerate(parsed_essay.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: @CAPS1 - ORG

Entity 2: @DATE1 - ORG

Entity 3: all day - DATE

Entity 4: @CAPS2 - ORG



In [16]:
token_text = [token.orth_ for token in parsed_essay]
token_pos = [token.pos_ for token in parsed_essay]

pd.DataFrame(zip(token_text, token_pos),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,Dear,ADJ
1,local,ADJ
2,newspaper,NOUN
3,",",PUNCT
4,I,PRON
5,think,VERB
6,effects,NOUN
7,computers,NOUN
8,have,VERB
9,on,ADP


In [17]:
token_lemma = [token.lemma_ for token in parsed_essay]
token_shape = [token.shape_ for token in parsed_essay]

pd.DataFrame(zip(token_text, token_lemma, token_shape),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,Dear,dear,Xxxx
1,local,local,xxxx
2,newspaper,newspaper,xxxx
3,",",",",","
4,I,-PRON-,X
5,think,think,xxxx
6,effects,effect,xxxx
7,computers,computer,xxxx
8,have,have,xxxx
9,on,on,xx


In [18]:
token_entity_type = [token.ent_type_ for token in parsed_essay]
token_entity_iob = [token.ent_iob_ for token in parsed_essay]

pd.DataFrame(zip(token_text, token_entity_type, token_entity_iob),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,Dear,,O
1,local,,O
2,newspaper,,O
3,",",,O
4,I,,O
5,think,,O
6,effects,,O
7,computers,,O
8,have,,O
9,on,,O


In [19]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_essay]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Dear,-11.536821,,,,,
1,local,-9.188298,,,,,
2,newspaper,-11.763153,,,,,
3,",",-3.454960,,Yes,,,
4,I,-3.791565,Yes,,,,
5,think,-6.180925,,,,,
6,effects,-9.838640,,,,,
7,computers,-10.650864,,,,,
8,have,-5.156485,Yes,,,,
9,on,-5.172736,Yes,,,,


In [20]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [21]:
# removing_stanford_nlp_groups_NER_from_stop_words(nlp)
adding_stanford_nlp_groups_NER_to_stop_words(nlp)

In [22]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all_essays.txt')

In [23]:
essays_set1_all_filepath = os.path.join(intermediate_directory, 'essay_set1_text_all.txt')

In [24]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(essays_set1_all_filepath, codecs, nlp):
            f.write(sentence + '\n')

CPU times: user 56.4 s, sys: 16.4 s, total: 1min 12s
Wall time: 1min 13s


In [25]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [26]:
for unigram_sentence in it.islice(unigram_sentences, 19, 42):
    print(' '.join(unigram_sentence))
    print('')

believe computer benefit way like talk friend website like facebook mysace

computer help find coordibate location able ourselfs million information

computer benefit help job plan house plan type page report job write

let wonder world technology

computer help life talk make friend line

people myspace facebooks aim benefit have conversation

people believe computer bad friend talk

fortunate have computer help school work social life friend

computer help find location coordibate million information online

internet lot know website help location coordinate like

use computer

suppose vacation

million information find internet

question computer

easily draw house plan computer hour hand ugly erazer mark garrentee find job drawing like

apple job worker write long paper like word essay job fit people know like write word non stopp hour

hav

computer

computer need lot adays

hope essay impact descion computer great machine work

day show mom use computer

say great invention sense

In [27]:
type(unigram_sentences)

gensim.models.word2vec.LineSentence

In [28]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

Train a phrase model for word pairs, let's apply it to the sentences data and explore the results

In [29]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 820 ms, sys: 37.6 ms, total: 858 ms
Wall time: 1.22 s


In [30]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [31]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = ' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

CPU times: user 1.49 s, sys: 8.55 ms, total: 1.5 s
Wall time: 1.52 s


In [32]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [35]:
for bigram_sentence in it.islice(bigram_sentences, 19, 42):
    print(' '.join(bigram_sentence))
    print('')

believe computer benefit way like talk friend website like facebook mysace

computer help find coordibate location able ourselfs million information

computer benefit help job plan house plan type page report job write

let wonder world technology

computer help life talk make friend line

people myspace facebooks aim benefit have conversation

people believe computer bad friend talk

fortunate have computer help school_work social life friend

computer help find location coordibate million information online

internet lot know website help location coordinate like

use computer

suppose vacation

million information find internet

question computer

easily draw house plan computer hour hand ugly erazer mark garrentee find job drawing like

apple job worker write long paper like word essay job fit people know like write word non stopp hour

hav

computer

computer need lot adays

hope essay impact descion computer great machine work

day show mom use computer

say great_invention sense

In [36]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [37]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 797 ms, sys: 54.5 ms, total: 852 ms
Wall time: 1.3 s


In [38]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [39]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = ' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

CPU times: user 1.43 s, sys: 6.51 ms, total: 1.44 s
Wall time: 1.45 s


In [40]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [43]:
for trigram_sentence in it.islice(trigram_sentences, 105, 145):
    print(' '.join(trigram_sentence))
    print('')

way quick_easy find board_game

reason computer help family family want kind activity need number online

internet family find number information

way family activity

computer bring family

reason computer good society education

information tip finger people want learn

day people try find tool computer mae people education

secondly computer way collage

arn't go school collage sick

miss click_button material need miss

lastly know classroom afford computer

child want computor hard

change

able learn

child lean different_country far_away

result computer improve way learn

reason computer helpful convience

world crazy talk will

computer offer ability confrence people communication work parttner

ability talk work light nation

reason computer conviente subssiquently computer conviente book_flight vacation rental

click_button go nice vacation

dear think computer negative_affect

people acess camputer daily america

people hour

mean people not exercise waste posibility physic

In [44]:
trigram_essays_all_filepath = os.path.join(intermediate_directory,
                                        'trigram_essays_all.txt')

In [52]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

#     with codecs.open(trigram_essays_all_filepath, 'w', encoding='utf_8') as f:
#         for sentence in lemmatized_sentence_corpus(essays_set1_all_filepath, codecs, nlp):
#             f.write(sentence + '\n')
    
    
    with codecs.open(trigram_essays_all_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_essay in nlp.pipe(line_review(essays_set1_all_filepath, codecs),
                                      batch_size=100, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_essays = [token.lemma_ for token in parsed_essay
                              if not punct_space_stop(token)]
            
            # apply the first-order and second-order phrase models
            bigram_essays = bigram_model[unigram_essays]
            trigram_essays = trigram_model[bigram_essays]
            
            # write the transformed review as a line in the new file
            trigram_essays = ' '.join(trigram_essays)
            f.write(trigram_essays + '\n')

CPU times: user 1min 2s, sys: 18.8 s, total: 1min 21s
Wall time: 1min 29s


In [56]:
print('Original:' + '\n')

for essay in it.islice(line_review(essays_set1_all_filepath, codecs), 101, 102):
    print(essay)

print('----' + '\n')
print('Transformed:' + '\n')

with codecs.open(trigram_essays_all_filepath, encoding='utf_8') as f:
    for essay in it.islice(f, 101, 102):
        print(essay)

Original:

Dear Local Newspaper, Experts should not be concerned about other people that stay on the computer for a while. Maybe people know what they are doing and have to stay on computers for a long time. They also might like to learn things that they never thought about learning. Most people try not to be overweight. Experts should Not try to have less people on the computer. One reason is that people can chat with one another. It can be friends, families, and more. About @PERCENT1 of people chat online with families or friends. Chatting can also help you like if there is an emergency. You can tell someone what is going on and call the cops. Without a doubt chatting online is a great way to use the computer during the time your on it. Another reason is that you can search anything you need to know. Maybe if you have to search something for school, the computer can help you. This can have positive effects on people. It can make someone a better person. You can't agree that getting t

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

In [109]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [59]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [60]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_essays = LineSentence(trigram_essays_all_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_essays)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 272 ms, sys: 11 ms, total: 283 ms
Wall time: 684 ms


Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [61]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [62]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for essay in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(essay)

In [63]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath, trigram_bow_generator(trigram_essays_all_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 339 ms, sys: 9.83 ms, total: 348 ms
Wall time: 448 ms


With the bag-of-words corpus, we're finally ready to learn our topic model from the essays. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 10 topics.

In [84]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [92]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=10,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 2.89 s, sys: 275 ms, total: 3.16 s
Wall time: 3.27 s


Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [93]:
def explore_topic(topic_number, topn=10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format('term', 'frequency') + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=10):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [106]:
explore_topic(topic_number=9)

term                 frequency

spend_time           0.026
outside              0.017
child                0.016
kid                  0.016
play                 0.014
not                  0.012
bad                  0.010
game                 0.008
problem              0.007
instead              0.007


In [107]:
topic_names = {0: 'kid_spend_time_website',
               1: 'spent_time_exercise_information',
               2: 'kid_game_look',
               3: 'internet_look_work',
               4: 'information_look_school',
               5: 'bad_outside_school',
               6: 'information_place_kid',
               7: 'lot_spend_time_effect',
               8: 'internet_look_game',
               9: 'spend_time_outside_child'}

## pickle stops this viz

In [111]:
# topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

# with open(topic_names_filepath, 'w') as f:
#     pickle.dump(topic_names, f)

In [108]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [112]:
# %%time

# # this is a bit time consuming - make the if statement True
# # if you want to execute data prep yourself.
# if 1 == 1:

#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
#                                               trigram_dictionary)

#     with open(LDAvis_data_filepath, 'w') as f:
#         pickle.dump(LDAvis_prepared, f)
        
# # load the pre-prepared pyLDAvis data from disk
# with open(LDAvis_data_filepath) as f:
#     LDAvis_prepared = pickle.load(f)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: write() argument must be str, not bytes

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [119]:
def get_sample_essay(essay_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(essay_set1_txt_filepath, codecs),essay_number, essay_number+1))[0]

In [122]:
def lda_description(essay_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the essay text with spaCy
    parsed_essay = nlp(essay_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_essay = [token.lemma_ for token in parsed_essay
                      if not punct_space_stop(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_essay = bigram_model[unigram_essay]
    trigram_essay = trigram_model[bigram_essay]
    
    # create a bag-of-words representation
    essay_bow = trigram_dictionary.doc2bow(trigram_essay)
    
    # create an LDA representation
    essay_lda = lda[essay_bow]
    
    # sort with the most highly related topics first
    essay_lda = sorted(essay_lda)
    
    for topic_number, freq in essay_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:25} {}'.format(topic_names[topic_number],round(freq, 3)))


In [125]:
sample_essay = get_sample_essay(2)
print(sample_essay)

Dear, @CAPS1 @CAPS2 @CAPS3 More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. Others have different ideas. A great amount in the world today are using computers, some for work and spme for the fun of it. Computers is one of mans greatest accomplishments. Computers are helpful in so many ways, @CAPS4, news, and live streams. Don't get me wrong way to much people spend time on the computer and they should be out interacting with others but who are we to tell them what to do. When I grow up I want to be a author or a journalist and I know for a fact that both of those jobs involve lots of time on time on the computer, one @MONTH1 spend more time then the other but you know exactly what @CAPS5 getting at. So what if some expert think people are spending to much time on the computer and not exercising, enjoying natures and interacting with family and friends

In [126]:
lda_description(sample_essay)

kid_spend_time_website    0.8539999723434448
internet_look_work        0.12999999523162842


## Word Vector Embedding with Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash; they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [129]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [ ]:
https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

Word2Vec(min_count=20,
                     window=5,
                     size=100,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [130]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    essay2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    essay2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        essay2vec.train(trigram_sentences)
        essay2vec.save(word2vec_filepath)
        
# load the finished model from disk
essay2vec = Word2Vec.load(word2vec_filepath)
essay2vec.init_sims()

print('{} training epochs so far.'.format(essay2vec.train_count))

ValueError: You must specify either total_examples or total_words, for proper job parameters updationand progress calculations. The usual value is total_examples=model.corpus_count.

#### Implementing Word2Vec From Scratch

In [41]:
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

In [42]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [43]:
norm_corpus = normalize_corpus(test_essay)
norm_corpus

array('dear local newspaper think effects computers people great learning skillsaffects give us time chat friendsnew people helps us learn globeastronomy keeps us troble thing dont think would feel teenager always phone friends ever time chat friends buisness partner things well theres new way chat computer plenty sites internet organization organization caps facebook myspace ect think setting meeting boss computer teenager fun phone rushing get cause want use learn countrysstates outside well computerinternet new way learn going time might think child spends lot time computer ask question economy sea floor spreading even dates youll surprise much heshe knows believe computer much interesting class day reading books child home computer local library better friends fresh perpressured something know isnt right might know child caps forbidde hospital bed driveby rather child computer learning chatting playing games safe sound home community place hope reached point understand agree comput

In [40]:
# from string import punctuation

# remove_terms = punctuation + '0123456789'

# norm_corpus = [[word.lower() for word in sent if word not in remove_terms] for sent in test_essay]
# norm_corpus = [' '.join(tok_sent) for tok_sent in norm_corpus]
# norm_corpus = filter(None, normalize_corpus(norm_corpus))
# norm_corpus = [tok_sent for tok_sent in norm_corpus if len(tok_sent.split()) > 2]

# print('Total lines:', len(test_essay))

# norm_corpus
# print('\nSample line:', test_essay[1])
# print('\nProcessed line:', norm_corpus[1])

[]

#### CBoW

In [47]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

tokenizer = text.Tokenizer()
type(norm_corpus)
# tokenizer.fit_on_texts(norm_corpus)
# word2id = tokenizer.word_index

# # build vocabulary of unique words
# word2id['PAD'] = 0
# id2word = {v:k for k, v in word2id.items()}
# wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_corpus]

# vocab_size = len(word2id)
# embed_size = 100
# window_size = 2 # context window size

# print('Vocabulary Size:', vocab_size)
# print('Vocabulary Sample:', list(word2id.items())[:10])

numpy.ndarray